In [ ]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

filepath1 = r"2021MCM_ProblemC_ Images_by_GlobalID.xlsx"
filepath2 = r"2021MCMProblemC_DataSet.xlsx"

data1 = pd.read_excel(filepath1)

# 对文件后缀名进行更改
for i in range(len(data1)):
    if(data1["FileName"][i].split(".")[-1] != "jpg"):
        data1["FileName"][i] = data1["FileName"][i].split(".")[0]+".jpg"
        data2 = pd.read_excel(filepath2)
        data = pd.merge(data1, data2, on="GlobalID")
        
class MyDataset(Dataset):
    def __init__(self, data_path):
        self.data_path = data_path
        self.img_list = os.listdir(self.data_path)

def __getitem__(self, index):
    img_title = self.img_list[index]
    # img_label = img_title.split('.')[0]
    data = pd.merge(data1, data2, on="GlobalID")
    ind = data[data["FileName"] == img_title].index.tolist()
    if(data["Lab Status"][ind[0]] == "Unprocessed"):
        img_label = np.array([0])
    elif(data["Lab Status"][ind[0]] == "Unverified"):
        img_label = np.array([1])
    elif(data["Lab Status"][ind[0]] == "Positive ID"):
        img_label = np.array([2])
    else:
        img_label = np.array([3])
        img_path = os.path.join(self.data_path, img_title)
        img = Image.open(img_path)
    img = np.array(img)
    # print(img.shape)
    return img, img_label


def __len__(self):
    return len(self.img_list)

if __name__ == '__main__':
    train_path = r'D:\Spyder\MCM_data'

In [ ]:
dataset = MyDataset(train_path)
# train_dataset=dataset.
train_size = int(dataset.__len__() * 0.8)
test_size = dataset.__len__() - train_size

train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=64)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)

# Input:[Batch, Channels, Height, Width]
for batch_idx, data in enumerate(train_loader):
    input, tar = data
    # print(type(input),'\t',input.shape)
    input = input.permute(0, 3, 1, 2)
    model = Net()
    output = model(input)
    _, predicted = torch.max(output.data, dim=1)
    # print((input),'\t',input.shape)
    print(predicted.size()[0])
    if batch_idx == 0:
        break

In [ ]:
import torch
from torch.utils.data import DataLoader
from time import *
import torch.nn as nn
import torch.nn.functional as F
# import torchsnooper
import torch.optim as optim
# from Dataset_make import MyDataset
import matplotlib.pyplot as plt

# prepare dataset
batch_size = 64
path = r'D:\Spyder\Data_cut'
dataset = MyDataset(path)
train_size = int(dataset.__len__() * 0.8)
test_size = dataset.__len__() - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
test_loader = DataLoader(test_dataset,shuffle=False,batch_size=batch_size)

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Input:Batch_size,3,600,450
        self.conv1 = torch.nn.Conv2d(3,10 , kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
        self.conv3=torch.nn.Conv2d(20,5,kernel_size=3)
        self.conv4=torch.nn.Conv2d(5,5,kernel_size=3)
        self.pooling = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(350, 4)
        # @torchsnooper.snoop()
        
    def forward(self, x):
    # 定义了每次执行的计算步骤。在所有的子类中都需要重写这个函数。
        # Flatten data from (n, 1, 28, 28) to (n, 784)
        batch_size = x.size(0)
        # print(x.size())
        x = F.relu(self.pooling(self.conv1(x)))
        # print(x.size())
        x = F.relu(self.pooling(self.conv2(x)))
        # print(x.size())
        x = F.relu(self.pooling(self.conv3(x)))
        # print(x.size())
        x = F.relu(self.pooling(self.conv4(x)))
        # print(x.size())
        x = x.view(batch_size, -1) # flatten
        # print(x.size())
        x = self.fc(x)
        # print(x.size())
        return x

model = Net()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.00001, momentum=0.5)

def train(epoch):
# (print(next(model.parameters()).device))
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader):
        inputs, target = data
        inputs = inputs.type(torch.FloatTensor)
        inputs=inputs.permute(0,3,1,2)
        inputs=inputs.cuda()
        # print(inputs.is_cuda)
        target = target.type(torch.LongTensor)
        target=target.cuda()
        # print(target.is_cuda)
        optimizer.zero_grad()#将module中的所有模型参数的梯度设置为0.
        # forward + backward + update
        outputs = model(inputs)
        temp=[]
    for i in range(len(target)):
        temp.append(target[i].item())
        # print(len(temp))
        temp=torch.LongTensor(temp)
        temp=temp.cuda()
        # print(temp.shape)
        loss = criterion(outputs, temp)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    if batch_idx % 10 ==9:
        print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss ))
        running_loss = 0.0
    torch.save(model.state_dict(), 'cnn_50.pkl')
    del inputs, target, outputs, loss
    # torch.cuda.empty_cache()

def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, target = data
            inputs = inputs.type(torch.FloatTensor)
            inputs=inputs.permute(0,3,1,2)
            target = target.type(torch.LongTensor)
            inputs=inputs.cuda()
            target=target.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, dim=1)
            total += target.size(0)
        temp=[]
        for i in range(len(target)):
            temp.append(target[i].item())
            temp=torch.LongTensor(temp)
            temp=temp.cuda()
            correct += (predicted == temp).sum().item()
            print('Accuracy on test set: %d %% [%d/%d]' % (100 * correct / total, correct, total))
            return correct / total
    if __name__ == '__main__':
        auc=[]
    begin_time = time()
    for epoch in range(10):
        train(epoch)
    auc.append(test())
    end_time = time()
    run_time = end_time - begin_time
    plt.plot(auc)
    font={'family' : 'Times New Roman','weight' : 'normal','size' : 30}
    font2 ={'family' : 'Times New Roman','weight' : 'normal','size' : 20}
    plt.xlabel("Epoch",font2)
    plt.ylabel("Auc",font2)
    plt.title("AUC Line",font)
    plt.grid
    plt.show()


In [ ]:
model=Net()
for batch_idx, data in enumerate(train_loader):
    inputs, target = data
    inputs = inputs.type(torch.FloatTensor)
    inputs=inputs.permute(0,3,1,2)
    outputs=model(inputs)